In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 데이터프레임 불러오기
data = pd.read_csv('/content/drive/MyDrive/Contest/공공조달/dataset/150_data_2.csv', index_col = 0)
data.fillna(data.mean(),inplace=True)
data.columns = ['Green_CNT','Gas','Water','Energy','Waste','Green_AMT','Energy_pr','Gas_reduce','Female_exe',
                'Disabled_pr','Small_CNT','Disabled_CNT','Env','Donation','Member','Female_member','Auditor','Female_RTO','Target' ]
# EDA에서 확인한 이상치 치환
def cut_outlier(df, columns):
    df=df.copy()
    for column in columns:
        q1=df[column].quantile(.25)
        q3=df[column].quantile(.75)
        iqr=q3-q1
        low=q1-1.5*iqr
        high=q3+1.5*iqr
        df.loc[df[column]<low, column]=low
        df.loc[df[column]>high, column]=high
    return df

data = cut_outlier(data, ['Waste', 'Energy'])

In [5]:
# OLS 회귀분석_인공요소
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import stats

y_data = data['Target']
X_data = data[['Small_CNT','Donation','Member','Female_member','Env','Disabled_CNT']]
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=2022)

art_model = sm.OLS(y_train, X_train)
art_result = art_model.fit()
art_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Target   R-squared (uncentered):                   0.280
Model:                            OLS   Adj. R-squared (uncentered):              0.242
Method:                 Least Squares   F-statistic:                              7.338
Date:                Tue, 15 Feb 2022   Prob (F-statistic):                    1.22e-06
Time:                        14:32:40   Log-Likelihood:                         -1806.6
No. Observations:                 119   AIC:                                      3625.
Df Residuals:                     113   BIC:                                      3642.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Small_CNT       172.8995     70.726      2.445      0.016      32.779     313.020
Donation         87.8461     32.294      2.720      0.008      23.867     151.825
Member          237.3863    105.655      2.247      0.027      28.065     446.708
Female_member  -366.6031    200.352     -1.830      0.070    -763.536      30.330
Env            4.413e+04   4.13e+04      1.069      0.287   -3.76e+04    1.26e+05
Disabled_CNT  -5.828e+04   3.07e+04     -1.900      0.060   -1.19e+05    2476.548
==============================================================================
Omnibus:                      135.955   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2525.548
Skew:                           4.126   Prob(JB):                         0.00
Kurtosis:                      24.006   Cond. No.                     1.70e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.7e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
# OLS 회귀분석_자연요소
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import stats

y_data = data['Target']
X_data = data[['Green_CNT','Water','Energy','Waste','Green_AMT','Energy_pr']]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=2022)

env_model = sm.OLS(y_train, X_train)
env_result = env_model.fit()
env_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 Target   R-squared (uncentered):                   0.722
Model:                            OLS   Adj. R-squared (uncentered):              0.708
Method:                 Least Squares   F-statistic:                              49.00
Date:                Tue, 15 Feb 2022   Prob (F-statistic):                    3.21e-29
Time:                        14:32:43   Log-Likelihood:                         -1749.9
No. Observations:                 119   AIC:                                      3512.
Df Residuals:                     113   BIC:                                      3529.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Green_CNT    607.4188    134.633      4.512      0.000     340.686     874.152
Water          0.3423      0.478      0.717      0.475      -0.604       1.288
Energy        26.7687     47.288      0.566      0.572     -66.917     120.455
Waste       -186.1809    440.687     -0.422      0.673   -1059.260     686.899
Green_AMT     26.2391      2.478     10.589      0.000      21.330      31.148
Energy_pr   -881.8774    208.049     -4.239      0.000   -1294.060    -469.695
==============================================================================
Omnibus:                      132.042   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2379.465
Skew:                           3.948   Prob(JB):                         0.00
Kurtosis:                      23.434   Cond. No.                     1.09e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.09e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""